In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow._api.v2 import data
import random

filePath = tf.keras.utils.get_file(
    "shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

text = open(filePath, 'rb').read().decode(encoding='utf-8')

vocab = sorted(set(text))

# charectar to integer representation (unqiue charecters and integer represetnation dictionary)
char2idx = {u: i for i, u in enumerate(vocab)}
# integer to charectar represetnation
idx2char = np.array(vocab)
# look up the idx representation of the charectars in the text and store it as an array of idx reprentations
textAsInt = np.array([char2idx[char] for char in text])

seqLength = 100
examplesPerEpoch = len(text)//(seqLength+1)

# split idx representations into indidvidual slices
charDataset = tf.data.Dataset.from_tensor_slices(textAsInt)

# Preparing the batches of data
sequences = charDataset.batch(seqLength+1, drop_remainder=True)

# splitting the labels and the input data


def splitInputTarget(chunk):
    inputText = chunk[:-1]
    targetText = chunk[1:]
    return inputText, targetText


# repreating the splitting function for all the sequences in the dataset
dataset = sequences.map(splitInputTarget)

BATCH_SIZE = 128
BUFFER_SIZE = 10000

# Prepare the batches and randomizing it
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

vocabSize = len(vocab)
print("VOCABSIZE =", vocabSize)
embeddingDim = 256
rnnUnits = 1024


def buildModel(vocabSize, embeddingDim, rnnUnits, batchSize):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(
            vocabSize, embeddingDim, batch_input_shape=[batchSize, None]),
        tf.keras.layers.GRU(rnnUnits, return_sequences=True,
                            stateful=True, recurrent_initializer="glorot_uniform"),
        tf.keras.layers.LSTM(rnnUnits, return_sequences=True,
                            stateful=True, recurrent_initializer="glorot_uniform"),
        tf.keras.layers.LSTM(rnnUnits, return_sequences=True,
                            stateful=True, recurrent_initializer="glorot_uniform"),
        tf.keras.layers.GRU(rnnUnits, return_sequences=True,
                            stateful=True, recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocabSize)
    ])
    return model


model = buildModel(vocabSize=len(vocab), embeddingDim=embeddingDim,
                   rnnUnits=rnnUnits, batchSize=BATCH_SIZE)

# Loss Function


def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss=loss,  metrics=["accuracy"])
checkpointDir = "./kaggle/working/training_checkpoint_Shakspeare_OptimizerChangedKaggle"
checkpointPrefix = os.path.join(checkpointDir, "chkpt_{epoch}")
checkpointCallback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpointPrefix, save_weights_only=True, monitor='val_accuracy')

EPOCHS = 50

# TRAIN
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpointCallback])



In [ ]:
# REBUILD FROM EXISTING TRAINING
model = buildModel(vocabSize, embeddingDim, rnnUnits, batchSize=1)

model.load_weights(tf.train.latest_checkpoint(checkpointDir))

model.build(tf.TensorShape([1, None]))

model.summary()

# generating text


def generateText(model, startString): 
    # of charectars generated
    numGenerate = 20000
    # convert start string into idx
    inputRep = [char2idx[s] for s in startString]
    inputRep = tf.expand_dims(inputRep, 0)

    txtGenerated = []
    # Handles randomness through a scale factor (smaller means predictable(increase for more randomness))
    tempreture = 1.0
    model.reset_states()
    for i in range(numGenerate):
        predictions = model(inputRep)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions/tempreture
        predictedID = tf.random.categorical(
            predictions, num_samples=1)[-1, 0].numpy()

        inputRep = tf.expand_dims([predictedID], 0)
        txtGenerated.append(idx2char[predictedID])
    return (startString + "".join(txtGenerated))

# prompts = ["ROMEO: ", "PUCK: ", "JULIET: ", "IAGO: ",
# "LEAR: ", "HAMLET: ", "MACBETH:", "hath ", "thou "]

print("PASSED")

word = ["Hath", "Thou", "Romeo", "Hamlet", "Macbeth", "Iago","Hath", "Young" ]
fileOut = open("./kaggle/working/ShakspeareGen4.txt", "w")
time = 6
number = 0
while time != 0:
    number  += 1
    seedword = word[number]
    fileOut.write(generateText(model, startString=seedword))
    print(time, " / ", "6", " - ", seedword)
    time = time-1

# Use sequencing for word variation

# for word in prompts:
    #filename = word + ".txt"
    #textOut = open(filename, "w")
    #textOut.write(generateText(model, startString=word))
    # textOut.close()
    # print("written")

#print(generateText(model, startString="Romeo: "))
